# Задание 2

In [2]:
from bs4 import BeautifulSoup
import pandas as pd

Используя режим разработчика я извлек из "https://www.w3schools.com/sql/trysql.asp?filename=trysql_select_all" html-страницы каждой таблицы и поместил их в текущую директорию. Используя библиотеку BeautifulSoup я проведу парсинг каждого html-документа и сформирую excel таблицы. Для этого я написал небольшую функцию.

In [183]:
def parse_html_to_excel(file_name: str):
    '''Функция принимает на вход название html-документа и формирует таблицу excel'''
    
    # создать пустую строку, в которую будет помещено содержимое html-документа
    # с помощью контекстного менеджера читаем файл
    html_text = ''
    with open(file_name + '.html', 'r') as f:
        html_text = f.read()
    
    # создаем soup объект пропарсеного html-документа
    soup = BeautifulSoup(html_text, 'html.parser')
    
    # находим таблицу
    table = soup.find('table', {'class': "w3-table-all notranslate"})
    
    # пустой список для названий колонок
    labels = []
    # ищем теги заголовков таблицы и помещаем их в список
    for trs in table.find_all('tr'):
        for ths in trs.find_all('th'):
            labels.append(ths.text)
    
    # пустой список для хранения списков данных каждого ряда
    values = []
    for trs in table.find_all('tr'):
        # ищем теги данных таблицы
        td_tags = trs.find_all('td')
        # с помощью генератора перебираем все значения колонок для конкретного ряда
        # и формируем из них список
        # для каждого значения убираем управляющую последовательность "\ха0",
        # которая появляется в процессе дешифровки
        td_vals = [value.text.replace(u'\xa0', '') for value in td_tags]
        values.append(td_vals)
    
    # на основе двух списков создаем data frame
    df = pd.DataFrame(values, columns=labels)  
    # назначаем колонку ID как индексную
    df.set_index(df.columns[0], inplace=True)
    # записываем data frame в файл excel
    df.to_excel(file_name + '.xlsx')

In [184]:
# создаем excel таблицы
parse_html_to_excel('customers')
parse_html_to_excel('categories')
parse_html_to_excel('employees')
parse_html_to_excel('products')
parse_html_to_excel('order_details')
parse_html_to_excel('shippers')
parse_html_to_excel('orders')
parse_html_to_excel('suppliers')

Таблицы сформированы, теперь можно приступать к решению задач

# 1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [525]:
# загружаем таблицу
customers = pd.read_excel('customers.xlsx', index_col=None)

# создаем логические фильтры
filter1 = customers['Country'] == 'Germany'
filter2 = customers['Country'] == 'France'
filter3 = customers['City'] == 'Madrid'

# фильтруем данные и удаляем все, что неподходят
customers = customers.where(filter1 | filter2 | filter3).dropna(axis=0)
# выбираем необходимые колонки и выводим 
query = customers[['CustomerName', 'Country', 'Address']]
query

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
5,Blauer See Delikatessen,Germany,Forsterstr. 57
6,Blondel père et fils,France,"24, place Kléber"
7,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
8,Bon app',France,"12, rue des Bouchers"
16,Drachenblut Delikatessend,Germany,Walserweg 21
17,Du monde entier,France,"67, rue des Cinquante Otages"
21,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
22,Folies gourmandes,France,"184, chaussée de Tournai"
24,Frankenversand,Germany,Berliner Platz 43


# 2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [524]:
# загружаем таблицу
customers = pd.read_excel('customers.xlsx', index_col=None)

# объединить по странам с агрегирующей функцией count
# выбрать только необходимые колонки
query = customers.groupby(['Country'], as_index=False).count()[['Country', 'CustomerID']]

# переименовать колонку CustomerID на OrderCount
query = query.rename(columns={'Country': 'Country', 'CustomerID': 'OrderCount'})

# отсортировать по убыванию и вывести первые три
query = query.sort_values(by='OrderCount', ascending=False).iloc[0:3]
query

,Country,OrderCount
19,USA,13
8,Germany,11
7,France,11


# 3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [523]:
# загружаем таблицы
orders = pd.read_excel('orders.xlsx', index_col=None, parse_dates=['OrderDate'])
shippers = pd.read_excel('shippers.xlsx', index_col=None)

# объединяем две таблицы
query = shippers.merge(orders, how='inner', on='ShipperID')

# сортируем по дате, выбираем нужные колонки и нужный ряд
query = query.sort_values(by='OrderDate', ascending=True)
query = query[['ShipperName', 'OrderDate']].iloc[9:10]
query

,ShipperName,OrderDate
130,Federal Shipping,1996-07-16


# 4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [522]:
# загружаем таблицы
order_details = pd.read_excel('order_details.xlsx', index_col=None)
products = pd.read_excel('products.xlsx', index_col=None)

# объединяем таблицы
merged_table = order_details.merge(products, how='inner', on='ProductID')

# создаем колонку со стоимостью
temporary_table = merged_table
temporary_table['Cost'] = temporary_table['Price'] * temporary_table['Quantity']

# группируем по номеру заказа с агрегирующей функцией суммирования
# сортируем по стоимости
# вычисляем самый дорогой заказ
temporary_table = temporary_table.groupby(['OrderID'], as_index=False).sum('Cost')[['OrderID', 'Cost']]
temporary_table = temporary_table.sort_values(by='Cost', ascending=False)
best_order = temporary_table.iloc[0, 0]

# создаем фильтр на айди самого дорогого заказа и выводим нужные колонки
filter1 = merged_table['OrderID'] == best_order
query = merged_table.where(filter1).dropna()[['ProductName', 'Price']]
query

,ProductName,Price
25,Mozzarella di Giovanni,34.8
74,Sir Rodney's Marmalade,81.0
92,Camembert Pierrot,34.0
490,Côte de Blaye,263.5


# 5. Какой товар больше всего заказывали по количеству единиц товара,  выведите его название и количество единиц в каждом из заказов.

In [521]:
# загружаем таблицы
order_details = pd.read_excel('order_details.xlsx', index_col=None)
products = pd.read_excel('products.xlsx', index_col=None)

# объединяем таблицы
merged_table = order_details.merge(products, how='inner', on='ProductID')

# группируем и считаем колличество единиц товара
# сортируем и вычисляем самый продаваемый товар
temporary_table = order_details
temporary_table = temporary_table.groupby(['ProductID'], as_index=False).sum('Quantity')[['ProductID', 'Quantity']]
temporary_table = temporary_table.sort_values(by='Quantity', ascending=False)
best_product = temporary_table.iloc[0,0]

# создаем фильтр на айди самого ходового продукта и выводим нужные колонки
filter1 = merged_table['ProductID'] == best_product
query = merged_table.where(filter1).dropna()[['OrderID','ProductName', 'Quantity']]
query

,OrderID,ProductName,Quantity
97,10253.0,Gorgonzola Telino,20.0
98,10272.0,Gorgonzola Telino,40.0
99,10273.0,Gorgonzola Telino,15.0
100,10325.0,Gorgonzola Telino,4.0
101,10335.0,Gorgonzola Telino,25.0
102,10342.0,Gorgonzola Telino,56.0
103,10356.0,Gorgonzola Telino,30.0
104,10359.0,Gorgonzola Telino,70.0
105,10363.0,Gorgonzola Telino,20.0
106,10374.0,Gorgonzola Telino,30.0


# 6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [406]:
# загружаем таблицы
order_details = pd.read_excel('order_details.xlsx', index_col=None)
products = pd.read_excel('products.xlsx', index_col=None)
suppliers = pd.read_excel('suppliers.xlsx', index_col=None)

# объединяем таблицы
merged_table = order_details.merge(products, how='inner', on='ProductID')
merged_table = merged_table[['OrderDetailID', 'OrderID', 'Quantity', 'SupplierID']]
merged_table2 = suppliers.merge(products, how='inner', on='SupplierID')
merged_table3 = merged_table.merge(merged_table2, how='inner', on='SupplierID',)

# группируем по айди поставщика суммируем кол-во товара
# сортируем и вычисляем айди пяти поставщиков
merged_table3 = merged_table3.groupby('SupplierID', as_index=False).sum('Quantity')
merged_table3 = merged_table3.sort_values(by='Quantity', ascending=False)
best_suppliers = merged_table3.iloc[0:5, 0]

# создаем фильтр на лучших поставщиков (по кол-ву заказов)
filter1 = suppliers['SupplierID'].isin(best_suppliers)

# выводим нужные колонки
query = suppliers.where(filter1).dropna()[['SupplierName', 'Country', 'ContactName', "Phone"]]
query

,SupplierName,Country,ContactName,Phone
6,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
7,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
11,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
13,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
14,Norske Meierier,Norway,Beate Vileid,(0)2-953010


# 7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [460]:
# загружаем таблицы
order_details = pd.read_excel('order_details.xlsx', index_col=None)
products = pd.read_excel('products.xlsx', index_col=None)
orders = pd.read_excel('orders.xlsx', index_col=None, parse_dates=['OrderDate'])
customers = pd.read_excel('customers.xlsx', index_col=None)
categories = pd.read_excel('categories.xlsx', index_col=None)

# объединяем таблицы
merged_table = categories.merge(products, how='inner', on='CategoryID')
merged_table2 = merged_table.merge(order_details, how='inner', on='ProductID')
merged_table3 = merged_table2.merge(orders, how='inner', on='OrderID')
merged_table4 = merged_table3.merge(customers, how='inner', on='CustomerID')

# создаем колонку со стоимостью
merged_table4['Cost'] = merged_table4['Price'] * merged_table4['Quantity']

# создаем фильтр на страну
filter1 = merged_table4['Country'] == 'Brazil'

# отсеиваем другие страны
table = merged_table4.where(filter1).dropna()

# группируем по айди категорий, сортируем по стоимости и вычисляем самую ходовую категорию в Бразилии
table2 = table.groupby('CategoryID', as_index=False).sum('Cost')
table2 = table2.sort_values(by='Cost', ascending=False)
best_category = table2.iloc[0,0]

# создаем фильтр на самую популярную категорию товаров
filter2 = table['CategoryID'] == best_category

# фильтруем данные и выводим нужные колонки
query = table.where(filter2).dropna()[['Country', 'CategoryName', 'Cost']]
query = query.groupby(['Country', 'CategoryName'], as_index=False).sum()
query

,Country,CategoryName,Cost
0,Brazil,Beverages,13690.0


# 8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [475]:
# загружаем таблицы
order_details = pd.read_excel('order_details.xlsx', index_col=None)
products = pd.read_excel('products.xlsx', index_col=None)
orders = pd.read_excel('orders.xlsx', index_col=None, parse_dates=['OrderDate'])
customers = pd.read_excel('customers.xlsx', index_col=None)

# объединяем таблицы
merged_table = order_details.merge(products, how='inner', on='ProductID')
merged_table2 = merged_table.merge(orders, how='inner', on='OrderID')
merged_table3 = merged_table2.merge(customers, how='inner', on='CustomerID')

# создаем колонку со стоимостью
merged_table3['Cost'] = merged_table3['Price'] * merged_table3['Quantity']

# создаем фильтр на страну
filter1 = merged_table3['Country'] == 'USA'

# отсеиваем другие страны
table = merged_table3.where(filter1).dropna()

# группируем по айди покупателя и сортируем по стоимости
table = table.groupby('CustomerID', as_index=False).sum('Cost')
table = table.sort_values(by='Cost', ascending=False)

# вычисляем разницу между самым дорогим и самым дешевым заказом
diff = (table['Cost'].max() - table['Cost'].min()).round()
print(f'Разница в стоимости между самым дорогим и самым дешевым заказом из США равна {diff}')

Разница в стоимости между самым дорогим и самым дешевым заказом из США равна 22079.0


# 9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [498]:
# загружаем таблицы
orders = pd.read_excel('orders.xlsx', index_col=None, parse_dates=['OrderDate'])
employees = pd.read_excel('employees.xlsx', index_col=None, parse_dates=['BirthDate'])

# конкатенируем колонки и убираем лишнее
employees['FullName'] = employees['LastName'] + ' ' + employees['FirstName']
employees = employees.drop(columns=['LastName', 'FirstName'])

# объединяем таблицы
merged_table = employees.merge(orders, how='inner', on='EmployeeID')

# группируем по имени и дате рождения, считаем все заказы
table = merged_table.groupby(['FullName', 'BirthDate'], as_index=False).count()

# сортируем по дате рождения и оставляем только первых трех
table = table.sort_values(by='BirthDate', ascending=False)[['OrderID', 'FullName']]
table = table.rename(columns={'OrderID': 'OrdersCount', 'FullName': 'FullName'})
table = table.iloc[0:3, :]
table

,OrdersCount,FullName
3,6,Dodsworth Anne
2,29,Davolio Nancy
6,31,Leverling Janet


# 10. Сколько банок крабового мяса всего было заказано.

In [505]:
# загружаем таблицы
order_details = pd.read_excel('order_details.xlsx', index_col=None)
products = pd.read_excel('products.xlsx', index_col=None)

# объединяем таблицы
merged_table = order_details.merge(products, how='inner', on='ProductID')

# создаем фильтр на крабовое мясо
filter1 = mereged_table['ProductName'] == 'Boston Crab Meat'

# фильтруем данные
table = merged_table.where(filter1).dropna()

# суммируем units крабового мяса и умножаем на 24, т.к. в описании товара указано что один юнит содержит 24 
# четырех-унцовые банки
crab_units = table['Quantity'].sum()
crab_tins = crab_units * 24
print(f'Было заказано {crab_tins} банок крабового мяса')

Было заказано 6144.0 банок крабового мяса
